## Data import and export in Scarf

In [1]:
%load_ext autotime

import scarf
scarf.__version__

'0.18.0'

time: 1.44 s (started: 2021-08-26 20:30:49 +02:00)


---
### 1) Fetch datsets from cloud repository

Scarf stores many single-cell datasets online on [OSF](https://osf.io/zeupv/). Herein datasets are stored in many different formats including MTX, 10x HDF5 and H5ad(anndata). These files can readily be downloaded using Scarf's `fetch_dataset` command.

To check which datasets are available to download, use the `show_available_datasets` function:

In [2]:
scarf.show_available_datasets()

annotations
baron_8K_pancreas_rnaseq
bastidas-ponce_4K_pancreas-d15_rnaseq
cao_2.1M_moca_rnaseq
cao_4.9M_fetal_rnaseq
cusanovich_81K_mouse_atacseq
domcke_721K_fetal_atacseq
hca_783K_blood_rnaseq
kang_14K_ifnb-pbmc_rnaseq
kang_15K_pbmc_rnaseq
lecun_60K_mnist_images
motifs
muraro_2K_pancreas_rnaseq
saunders_110K_brain_rnaseq
segerstolpe_2K_pancreas_rnaseq
tenx_1.3M_brain_rnaseq
tenx_10K_pbmc-v1_atacseq
tenx_3K_pbmc_multiome-gex-atac
tenx_5K_pbmc_rnaseq
tenx_8K_pbmc_citeseq
xin_1K_pancreas_rnaseq
zalando_60K_fmnist_images
zeisel_161K_nervous_rnaseq
zheng_69K_pbmc_rnaseq
time: 6.81 s (started: 2021-08-26 20:30:51 +02:00)


**Naming format**: Datasets are named using this rule: \<author\>\_\<number of cells\>\_\<cell/tissue type or species\>\_\<single-cell method>

Now using any of these dataset names we can download the dataset of our choice:

In [3]:
# This dataset is in Cellranger (10x) HDF5 format.
scarf.fetch_dataset(
    dataset_name='tenx_10K_pbmc-v1_atacseq',
    save_path='./scarf_datasets'
)

time: 5.75 s (started: 2021-08-26 20:30:58 +02:00)


The above dataset gets saved under the directory `scarf_datasets` in our current working directory. You can modify `save_path` parameter to save data in location of your choice. The dataset above was downloaded in 10x's HDF5 format. Let download few more datasets that are in differnet file formats.

In [4]:
# This dataset is in MTX format along with barcodes and features TSV files.
scarf.fetch_dataset(
    dataset_name='xin_1K_pancreas_rnaseq',
    save_path='./scarf_datasets'
)

time: 9.94 s (started: 2021-08-26 20:31:03 +02:00)


In [5]:
# This dataset is in H5ad (anndata) format.
scarf.fetch_dataset(
    dataset_name='bastidas-ponce_4K_pancreas-d15_rnaseq',
    save_path='./scarf_datasets'
)

time: 4.28 s (started: 2021-08-26 20:31:13 +02:00)


---
### 2) Conversion to Scarf's Zarr format file

Scarf stores data as dense, compressed chunks in Zarr file format. `scarf.readers` and `scarf.writers` modules contain classes that allow reading many different file formats and convert them to Zarr. There are often complementary reader and writer classes. Let's explore them below.

#### From 10x's HDF5 file format

In [6]:
# Change file_type to 'rna' in case of sc-RNA-seq or CITE-Seq
reader = scarf.CrH5Reader(
    'scarf_datasets/tenx_10K_pbmc-v1_atacseq/data.h5'
)

# change value of `zarr_fn` to your choice of filename and path
writer = scarf.CrToZarr(
    reader,
    zarr_fn='scarf_datasets/pbmc_atac.zarr'  
)  
writer.dump()

  0%|                                                                                                         …

time: 22.5 s (started: 2021-08-26 20:31:18 +02:00)


#### From 10x's (Cellranger) MTX file format

`scarf.CrDirReader` class reads MTX files generated by Cellranger pipeline. `CrDirReader` stands for 'Cellranger directory reader'. Once read in, the data can be dumped into Zarr format using `scarf.CrToZarr` class. Following is an example of how to do this conversion:

In [7]:
 # Note here we only give name of directory containing MTX file (along with barcodes and features file)
reader = scarf.CrDirReader(
    'scarf_datasets/xin_1K_pancreas_rnaseq'
)

# change value of `zarr_fn` to your choice of filename and path
writer = scarf.CrToZarr(
    reader, 
    zarr_fn='scarf_datasets/xin_1K.zarr'
)
writer.dump()

  0%|                                                                                                         …

time: 5.88 s (started: 2021-08-26 20:31:40 +02:00)


#### From Anndata H5ad file format

In [8]:
 # Note here we only give name of directory containing MTX file (along with barcodes and features file)
reader = scarf.H5adReader(
    'scarf_datasets/bastidas-ponce_4K_pancreas-d15_rnaseq/data.h5ad', 
    cell_ids_key = 'index',               # Where Cell/barcode ids are saved under 'obs' slot
    feature_ids_key = 'index',            # Where gene ids are saved under 'var' slot
    feature_name_key = 'gene_short_name'  # Where gene names are saved under 'var' slot
)  

# change value of `zarr_fn` to your choice of filename and path
writer = scarf.H5adToZarr(
    reader,
    zarr_fn='scarf_datasets/differentiating_pancreatic_cells.zarr'
)
writer.dump()

INFO: No value provided for assay names. Will use default value: 'RNA'


Reading attributes from group obs:   0%|                                                                      …

Reading attributes from group var:   0%|                                                                      …

  0%|                                                                                                         …

time: 2.75 s (started: 2021-08-26 20:31:46 +02:00)


Conversion from [Loom](https://loompy.org/) file formats is also supported using `scarf.LoomReader` and `scarf.LoomToZarr` which can be used in similar fashion as other readers and writers.

---
### 3) Exporting to data from Zarr file format

#### To Cellranger (10x) MTX format

In [9]:
ds = scarf.DataStore('scarf_datasets/differentiating_pancreatic_cells.zarr')

(RNA) Computing nCells and dropOuts:   0%|                                                                    …

(RNA) Computing nCounts:   0%|                                                                                …

(RNA) Computing nFeatures:   0%|                                                                              …

(RNA) Computing RNA_percentMito:   0%|                                                                        …

(RNA) Computing RNA_percentRibo:   0%|                                                                        …

time: 2.56 s (started: 2021-08-26 20:31:49 +02:00)


In [10]:
scarf.writers.to_mtx(
    assay=ds.RNA,
    mtx_directory='scarf_datasets/diff_pancreas'
)

  0%|                                                                                                         …

time: 21.5 s (started: 2021-08-26 20:31:51 +02:00)


#### To H5ad format

Conversion to H5ad is the preferred mode as it runs much faster and produces files with smaller footprints. Updates are underway to include all the data from Zarr file like UMAP, PCA and graph, into anndata.

In [11]:
ds = scarf.DataStore('scarf_datasets/differentiating_pancreatic_cells.zarr')

time: 31 ms (started: 2021-08-26 20:32:13 +02:00)


In [12]:
scarf.writers.to_h5ad(
    assay=ds.RNA,
    h5ad_filename='scarf_datasets/diff_pancreas.h5ad'
)

  0%|                                                                                                         …

time: 3.38 s (started: 2021-08-26 20:32:13 +02:00)


---
That is all for this vignette.